In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
########################### 답변 간이로 확인 ###########################


import os
import numpy as np
import torch

import sys
sys.path.append('/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM')

# from model.kogpt2 import DialogKoGPT2
from kogpt2_transformers import get_kogpt2_tokenizer

import kss

root_path = '/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM'
# data_path = f"{root_path}/data/wellness_dialog_for_autoregressive_train.txt"
save_ckpt_path = f"{root_path}/checkpoint/kogpt2-wellnesee-auto-regressive_hyper.pth"
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

import torch.nn as nn
from kogpt2_transformers import get_kogpt2_model

# model
class DialogKoGPT2(nn.Module):
  def __init__(self):
    super(DialogKoGPT2, self).__init__()
    self.kogpt2 = get_kogpt2_model()

  def generate(self,
               input_ids,
               do_sample=True,
               max_length= 60,
               top_p=1,
               top_k=50,
               temperature= 0.6,
               no_repeat_ngram_size=2,
               num_return_sequences=1,
               early_stopping=True,
               ):
    return self.kogpt2.generate(input_ids,
               do_sample=do_sample,
               max_length=max_length,
               top_p = top_p,
               top_k=top_k,
               temperature=temperature,
               no_repeat_ngram_size= no_repeat_ngram_size,
               num_return_sequences=num_return_sequences,
               early_stopping = early_stopping,
              )

  def forward(self, input, labels = None):
    if labels is not None:
      outputs = self.kogpt2(input, labels=labels)
    else:
      outputs = self.kogpt2(input)

    return outputs


model = DialogKoGPT2()
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

tokenizer = get_kogpt2_tokenizer()

sent = 1
while sent:
  sent = input('Question: ')  # '요즘 기분이 우울한 느낌이에요'
  if sent != '종료':  # 종료가 아니면 진행
    tokenized_indexs = tokenizer.encode(sent)  # 입력받은 문장을 토크나이저로 인코드함.
    input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0)  #
    sample_output = model.generate(input_ids=input_ids)
    out_sent = tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True)
    # print(out_sent)

    # kss 사용해서 text split
    sentences = kss.split_sentences(out_sent)
    selected_fir_sentence = sentences[0]
    selected_sec_sentence = sentences[1] if sentences[0] != sentences[1] else ''

    print("Answer: " + selected_fir_sentence + selected_sec_sentence)
    print(100 * '-')

  else:
        break


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Question: 착잡해


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: 저는 너무 외로울 때면 말랑한 무언가를 만지고는 해요.단순한 행동에 집중하면 기분이 약간은 나아질 거예요.
----------------------------------------------------------------------------------------------------
Question: 고마워. 하지만 힘들어


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Answer: 네, 그렇군요.듣고 있으니 계속 말씀하세요.
----------------------------------------------------------------------------------------------------
Question: ㅋㅋㅋㅋㅋ


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: 으휴, 제가 대신 혼내주고 싶네요!우리집 가정사를 다 되어주시네요.
----------------------------------------------------------------------------------------------------
Question: ㅠㅠㅠㅠㅠㅠㅠ


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: index out of range in self